# place cell tutorial

### Introduction

Let's say we performed an ephys recording from the hippocampus while an animal ran through a linear track. We want to identify place cells. To do this, we need to get from the raw spiking activity from all neurons to the spatial firing rate maps for spatially modulated cells. 

![title](images/place_cell_tutorial-02.png)

How to we get from the raw data to place cell maps? Let's walk through the steps below.

### Setup

Before we get started, since we're working in python we need to import the relevant scientific libraries to help us perform our analysis. We're also going to define a few functions that we will use later on, but we don't need to go into the details of here. (Note: to execute a block of code, press the shift + enter keys at the same time)

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal

%matplotlib inline

In [ ]:
#define variables and functions that we will use
n_positionbins = 50
samp_rate = 1000 #Hz

def plot_raw_data(df):
    '''
    This function plots the raw spike trains and mouse trajectories from a given dataframe
    '''
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,4))
    fig.suptitle('Raw data')

    #plot position data
    ax1.plot(df.behavior.position)
    ax1.set_ylabel('Position (cm)')
    ax1.set_xlabel('Samples')
    ax1.set_title('Mouse trajectories')

    #plot spike trains
    cmap = plt.get_cmap("tab10")
    for ineuron in range(np.shape(df.neurons)[1]):
        ax2.eventplot(np.where(df.neurons.iloc[:,ineuron])[0],lineoffsets = ineuron, colors = cmap(ineuron))
    ax2.set_xlabel('Samples')
    ax2.set_ylabel('Neurons')
    ax2.set_title('Spike trains');

def select_data_when_moving(df,speed_threshold):
    '''
    This function replaces neuron spike train values with nan if the speed is below a certain threshold
    '''
    #calc speed
    speed = df.behavior.position.diff()*samp_rate #cm/sec
    speed[speed<0] = np.nan #replace times when mouse teleported with nan

    #replace data points when not moving with nans
    mask = speed < speed_threshold
    df.loc[mask,"neurons"] = np.nan
    return df

def get_pos_map(df,step,max_pos):
    '''
    This function gets mapping of samples from data frame to position bin given step size and position of interest
    '''
    acc_map = np.empty(np.shape(df.behavior.position))
    for idx, ibin in enumerate([[i, i+step] for i in range(0, max_pos, step)]): 
        pos_inds = np.where((df.behavior.position >= min(ibin)) & (df.behavior.position < max(ibin)))[0]
        acc_map[pos_inds] = idx
    return acc_map
        
def get_spike_count_map(spike_train,acc_map):
    '''
    This function bins and sums spike train counts to a given position mapping
    '''
    st = spike_train.values
    keep = ~np.isnan(st)
    spike_counts_map = np.bincount(acc_map[keep].astype(int),st[keep])
    return spike_counts_map

def smooth_with_gaussian(orig_vect,std):
    '''
    This function smooths a vector using a gaussian window with a given std
    '''
    #convert data into usable form if is pd.Series
    if isinstance(orig_vect,pd.Series):
        orig_vect = orig_vect.values

    #smooth data
    pad_len = np.round(std*2.5)
    pad_vect = np.concatenate([np.ones(int(pad_len))*orig_vect[0], orig_vect, np.ones(int(pad_len))*orig_vect[-1]])
    gauss_window = signal.gaussian(std*7, std=std)
    vect_smooth_long = np.convolve(gauss_window,pad_vect,'same')
    vect_smooth = vect_smooth_long[int(pad_len):int(pad_len+len(orig_vect))]
    return vect_smooth

def plot_rate_maps(rate_map_norm):
    '''
    This function plots the firing rate maps
    '''
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,4))
    fig.suptitle('Firing rate maps')

    #heatmap
    pos_labels = np.round(np.linspace(0,max_pos,6))
    pos_loc = np.round(np.linspace(0,max_pos/2,6))
    h = sns.heatmap(rate_map_norm.T,
                    xticklabels=pos_labels,
                    ax=ax2,
                    cbar_kws={'label': 'Firing rate (spikes/s)'},
                    vmin=0,
                    vmax=1)
    h.set_xticks(pos_loc)
    h.set_xlabel('Position (cm)')
    h.set_ylabel('Neurons')

    #line plot 
    ax1.plot(pos_vect,rate_map_norm)
    ax1.set_xlabel('Position (cm)')
    ax1.set_ylabel('Firing rate (spikes/s)');

### Section 1 - viewing and cleaning our raw data

Let's start with the raw(-ish) data. For each neuron, we have a vector of spike counts for each sample. We also have the time points and positions for the whole recording. First, we will load up the data structure and visualize it.

In [ ]:
df = pd.read_pickle('toy_data.pkl') 
df.head()

And when we plot our data, it looks something like this:

In [ ]:
plot_raw_data(df)

First we want to clean our data and make sure we're only using times when the animal is running or moving above a certain speed threshold. When the animals is still, place cells from anywhere in the track are reactivated during sharp-wave ripples and this information can contaminate our place field analysis. To do this, we will replace the data when the animal is below a certain speed threshold with nan values. 

In [ ]:
speed_threshold = 10 #cm/s
df_when_moving = select_data_when_moving(df,speed_threshold)
df_when_moving.head()

### Section 2 - transforming data in terms of position

Now we want to get our data put in terms of position and make a *spike counts map*. What this means is that for each neuron, we want to know how many spikes occurred in each position in the track. To do this we will count the number of spikes in each position bin. So if the track is 100m long, we will first count the spikes in the first 1-5m from the start of the track, then  6-10m from the start, then 11-15m, etc. 

In [ ]:
#get map of which samples belong to which position bins for the whole recording
max_pos = round(max(df.behavior.position))
step = round(max_pos/n_positionbins)
pos_vect = np.round(np.linspace(0,max_pos,n_positionbins))
accum_map = get_pos_map(df,step,max_pos)

#apply mapping to each neuron
spike_counts_map = df.neurons.apply(lambda x: get_spike_count_map(x,accum_map))

#plot the spatial maps
plt.plot(pos_vect,spike_counts_map)
plt.xlabel('Position (cm)')
plt.ylabel('Spike counts')
plt.title('Spatial maps for neurons');

Now we have a *spike counts map.* But that won't be enough information to get our place cells, because we also need to know how long the animal spent in each position. If the animal spent a lot of time consuming reward at the end of the track, there will be a LOT of spikes in that location, but that could be due entirely to the time spent in that location. So we now want to get an *occupancy map* to account for time.

In [ ]:
#insert code here to get occupancy map
time_spent = df.behavior.time.diff()
keep = ~np.isnan(time_spent)
occupancy_map = np.bincount(accum_map[keep].astype(int),time_spent[keep])

#plot the occupancy map
plt.plot(pos_vect,occupancy_map)
plt.xlabel('Position (cm)')
plt.ylabel('Time (s)')
plt.title('Occupancy map');

### Section 3 - generating our spatial maps
We now have a spike counts map and an occupancy map. The raw data can be a bit messy across different position bins, so to clean this up and improve our signal to noise ratio, we will smooth the maps. After smoothing the maps, we will combine them in order to get our final spatial map (spike counts normalized by occupancy) for each neuron.  

In [ ]:
# smooth the maps
bin_range = 1
spike_counts_map_smooth = spike_counts_map.apply(lambda x: smooth_with_gaussian(x,bin_range))
occupancy_map_smooth = smooth_with_gaussian(occupancy_map, bin_range)
p = plt.plot(pos_vect,occupancy_map_smooth)
plt.xlabel('Position (cm)')
plt.ylabel('Time (s)')
plt.title('Occupancy map');

In [ ]:
#make rate maps 
rate_map = spike_counts_map_smooth.apply(lambda x: x/occupancy_map_smooth)
rate_map_norm = rate_map.apply(lambda x: x/max(x))

#plot rate maps
plot_rate_maps(rate_map_norm)

### Section 4 - identifying place cells
Now we have a spatial map for every neuron, but that doesn't mean that every neuron is spatially modulated. What we want to do next is identify place cells, neurons that *are* spatially modulated. There are many different ways to do this (spatial info, reliability, firing rate thresholds, etc.), but for the purposes of this tutorial we will a  simple criteria of a peak value that is a certain number of std above the mean

In [ ]:
#select place cell data that meets threshold criteria
std_threshold = 1.5 #std above the mean firing rate
threshold_vals = std_threshold*rate_map_norm.std()+rate_map_norm.mean() 
place_cell_mask = rate_map_norm.max()>threshold_vals 
place_cell_maps = rate_map_norm[place_cell_mask.index[place_cell_mask]]

#plot final place cells
plot_rate_maps(place_cell_maps)

### Summary

You now have identified place cells! See the full analysis process we walked through below.

![title](images/place_cell_tutorial-01.png)

### Section 5 - Future directions and other notes
You can use these place cells in other analyses, such as place cell reactivation and theta phase precession. You can also look at the spatial map features in more detail and quantify things such as  place fields, peak firing rates, spatial info, etc. 

For real data, you will need to account for some other things such as empty position bins with zero occupancy or spatial areas with multiple dimensions. The criteria you apply to classify place vs. non-place cells is also important because the output can change depending on which criteria you choose to apply.